In [1]:
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from fairseq.models.roberta import RobertaModel

In [2]:
roberta = RobertaModel.from_pretrained(
    '../',
    checkpoint_file='checkpoints/checkpoint_best.pt',
    data_name_or_path="./data"
)

roberta.eval()  # disable dropout

loading archive file ../
| [input] dictionary: 50265 types
| [label] dictionary: 9 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm((768

In [3]:
def replace_query_token(sentence):
    "Small utility function to replace a sentence with `_die_` or `_dat_` with the proper RobBERT input."
    tokens = nltk.word_tokenize(sentence)
    tokens_swapped = nltk.word_tokenize(sentence)
    for i, word in enumerate(tokens):
        if word == "_die_":
            tokens[i] = "die"
            tokens_swapped[i] = "dat"

        elif word == "_dat_":
            tokens[i] = "dat"
            tokens_swapped[i] = "die"

        elif word == "_Dat_":
            tokens[i] = "Dat"
            tokens_swapped[i] = "Die"

        elif word == "_Die_":
            tokens[i] = "Die"
            tokens_swapped[i] = "Dat"


        if word.lower() == "_die_" or word.lower() == "_dat_":
            results = TreebankWordDetokenizer().detokenize(tokens)
            results_swapped = TreebankWordDetokenizer().detokenize(tokens_swapped)

            return "{} <sep> {}".format(results, results_swapped)            


In [4]:
sentence = "Vervolgens zullen we de gebruikelijke procedure volgen, _dat_ wil zeggen dat we een voorstander en een tegenstander van dit verzoek het woord zullen geven."

tokens = roberta.encode(replace_query_token(sentence))

print("Correct sentence: ", roberta.predict('sentence_classification_head', tokens).argmax().item() == 1)

Correct sentence:  True


In [5]:
sentence = "Vervolgens zullen we de gebruikelijke procedure volgen, _die_ wil zeggen dat we een voorstander en een tegenstander van dit verzoek het woord zullen geven."

tokens = roberta.encode(replace_query_token(sentence))

print("Correct sentence: ", roberta.predict('sentence_classification_head', tokens).argmax().item() == 1)

Correct sentence:  False


In [6]:
sentence = "Daar loopt _die_ meisje."
tokens = roberta.encode(replace_query_token(sentence))

#print(roberta.predict('sentence_classification_head', tokens))
print("Correct sentence: ", roberta.predict('sentence_classification_head', tokens).argmax().item() == 1)

Correct sentence:  False
